In [13]:
import pandas as pd

import wmfdata as wmf

In [5]:
views = wmf.spark.run("""
SELECT
    SUM(CAST(is_pageview AS INT)) AS pageviews,
    SUM(CAST(NOT is_pageview AS INT)) AS previews,
    year,
    month,
    day,
    access_method,
    x_analytics_map["wprov"] = "wppw1t" AS has_touch_parameter,
    parse_url(referer, 'HOST') AS referer_host
FROM wmf.webrequest
WHERE
    x_analytics_map["wprov"] IN ('wppw1', 'wppw1t')
    AND webrequest_source = 'text'
    AND year = 2022
GROUP BY
    year,
    month,
    day,
    access_method,
    x_analytics_map["wprov"] = "wppw1t",
    parse_url(referer, 'HOST')
""", session_type="yarn-large")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
22/01/10 01:22:51 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


In [17]:
views["time"] = pd.to_datetime(views[["year", "month", "day"]])

In [39]:
touch_parameter_previews = views.query("has_touch_parameter and referer_host not in ('0.0.0.0', 'localhost')")
touch_parameter_previews

,pageviews,previews,year,month,day,access_method,has_touch_parameter,referer_host,time
8,0,1,2022,1,5,desktop,True,rightforeducation.org,2022-01-05
49,0,1,2022,1,5,desktop,True,sanremo-festival.de,2022-01-05
58,0,3,2022,1,6,desktop,True,mindfull.monster,2022-01-06
132,0,4,2022,1,8,desktop,True,buildasmarterhome.com,2022-01-08
138,0,1,2022,1,8,desktop,True,hedendaagsesieraden.nl,2022-01-08
149,0,1,2022,1,9,desktop,True,afanporsaber.com,2022-01-09
226,0,1,2022,1,8,desktop,True,college-emile-durkheim.fr,2022-01-08
241,0,5,2022,1,8,desktop,True,pk-andrianakis.lima-city.de,2022-01-08
250,0,3,2022,1,8,desktop,True,www.gomcqs.net,2022-01-08
278,0,1,2022,1,9,desktop,True,www.gomcqs.net,2022-01-09


In [40]:
touch_parameter_previews.groupby("time")["previews"].sum()

time
2022-01-04     9
2022-01-05     2
2022-01-06     3
2022-01-08    14
2022-01-09     2
Name: previews, dtype: int64

In [43]:
(
    views
    .groupby(["time", "has_touch_parameter"])
    ["previews"]
    .sum()
    .to_frame()
    .unstack()
    .fillna(0)
    .applymap(int)
    .rename_axis()
    .droplevel(0, axis="columns")
)

has_touch_parameter,False,True
time,,
2022-01-01,301,0
2022-01-02,317,0
2022-01-03,301,10
2022-01-04,547,30
2022-01-05,579,2
2022-01-06,466,3
2022-01-07,424,0
2022-01-08,392,14
2022-01-09,502,2
